# Replication study

Replicate the DeepAR results for:

- Rangapuram, Syama Sundar, et al. "Deep state space models for time series forecasting." Advances in Neural Information Processing Systems. 2018.

## Data

Among the data there are two publicly available datasets, namely `electricity` and `traffic`

- `electricity` - Hourly time series of electricity consumption of 370 customers
- `traffic` - Hourly occupancy rates (between 0 and 1) of 963 car lanes of San Francisco bay area freeways

Both datasets exhibit hourly and daily seasonal patterns.

### Benchmark Methods 

- `auto.arima` - automatic ARIMA framework from R *forecast* package
- `ets` - automatic expoential smoothing, also from R *forecast* package

Benchmark methods are compared to `DeepAR` (this notebook) and `DeepState` (next notebook). For the paper, the built-in algorithm of the AWS SageMaker is used. I will see if the `gluonts.model.deepar` implementation is comparable.  

### RNN-based methods 

Inputs for `DeepAR`and `DeepState`: 

- Independent feature representing the category (i.e. the index)
- time-based features like hour-of-the-day, day-of-the-week, day-of-the-month
- `DeepState` - Size of state space model directly depends on the granularity of the time series which determines the number of seasons. 
 - For hourly data - hour-of-the day (24 seasons), day-of-the-week (7 seasons) models and hence latent dimension is 31
- Train each method on all time series of these datasets but vary the size of the training range $T_i \in {14, 21, 28}$ days.
- Evaluate on the next $\tau = 7$ days using *p50* and *p90* quantile loss.

#### Imports

In [1]:
# standard imports
import numpy as np
# import pandas as pd

# json
# import json

# gluon data
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes

# gluon imports
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

# model imports
from gluonts.model.deepar import DeepAREstimator

import mxnet as mx
from pprint import pprint

# seeds
mx.random.seed(42)
# np.random.seed(42)


INFO:root:Using CPU


In [2]:
dataset = get_dataset("electricity")

# Inspect data
print(len(list(dataset.train)))
print(len(list(dataset.test)))

entry = list(dataset.test)[0]

print("entry:\n", entry)
print("metadata: \n", dataset.metadata)
print("prediction_length:n", dataset.metadata.prediction_length)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\electricity.


321
2247
entry:
 {'start': Timestamp('2012-01-01 00:00:00', freq='H'), 'target': array([14., 18., 21., ...,  8.,  8.,  6.], dtype=float32), 'feat_static_cat': array([0]), 'source': SourceContext(source=Span(path=WindowsPath('C:/Users/TM/.mxnet/gluon-ts/datasets/electricity/test/data.json'), line=1), row=1)}
metadata: 
 MetaData freq='1H' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='321'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=24
prediction_length:n 24


In [3]:
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,      # default: 100
        batch_size=32,      # default: 32
        num_batches_per_epoch=50,      #default: 50
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 19863
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:26<00:00,  1.90it/s, avg_epoch_loss=6.69]
INFO:root:Epoch[0] Elapsed time 26.269 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.689123
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:18<00:00,  2.67it/s, avg_epoch_loss=5.88]
INFO:root:Epoch[1] Elapsed time 18.730 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=5.882805
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:15<00:00,  3.22it/s, avg_epoch_loss=5.71]
INFO:root:Epoch[2] Elapsed time 15.520 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=5.711921
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:19<00:00,  2.57it/s, avg_epoch_loss=5.58]
INFO:root:Epoch[3] Elapsed time 19.463 seconds
INFO:root:Epoch[3] Evaluation metric 'epoch_loss'=5.583702
INFO:root:Epoch[4] Learning rate is 0.

100%|██████████| 50/50 [00:13<00:00,  3.81it/s, avg_epoch_loss=5.17]
INFO:root:Epoch[73] Elapsed time 13.120 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.174998
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.59it/s, avg_epoch_loss=5.1] 
INFO:root:Epoch[74] Elapsed time 13.939 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.101086
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.74it/s, avg_epoch_loss=5.03]
INFO:root:Epoch[75] Elapsed time 13.387 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.031799
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 50/50 [00:16<00:00,  3.10it/s, avg_epoch_loss=5.1] 
INFO:root:Epoch[76] Elapsed time 16.155 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.096562
INFO:root:Epoch[77] Learning rate is 0.00025
100%|██████████| 50/50 [00:15<00:00,  3.21it/s, avg_epoch_loss=5.08]
INFO:root:Epoch[77] Elapsed time 15.560 sec

{'Coverage[0.1]': 0.08436645686311892,
 'Coverage[0.2]': 0.15595397037319628,
 'Coverage[0.3]': 0.23161040116981485,
 'Coverage[0.4]': 0.3067582173056144,
 'Coverage[0.5]': 0.3899167143492924,
 'Coverage[0.6]': 0.47510967003623866,
 'Coverage[0.7]': 0.5765782948693492,
 'Coverage[0.8]': 0.6845317566278868,
 'Coverage[0.9]': 0.8027210884353791,
 'MAE_Coverage': 0.08805038110778993,
 'MASE': 0.8330967249588235,
 'MSE': 5258773.763251051,
 'MSIS': 10.944287886928509,
 'ND': 0.07437302412764038,
 'NRMSE': 0.7106490787782086,
 'QuantileLoss[0.1]': 1413345.8631078955,
 'QuantileLoss[0.2]': 2313039.9757389817,
 'QuantileLoss[0.3]': 2971822.365715612,
 'QuantileLoss[0.4]': 3449134.6758742547,
 'QuantileLoss[0.5]': 3774883.954206478,
 'QuantileLoss[0.6]': 3937504.8743052958,
 'QuantileLoss[0.7]': 3899732.362636425,
 'QuantileLoss[0.8]': 3596312.208871216,
 'QuantileLoss[0.9]': 2882575.0383782685,
 'RMSE': 2293.2016403384705,
 'abs_error': 3774883.906194687,
 'abs_target_mean': 3226.911437472181

In [4]:
dataset = get_dataset("traffic")

# Inspect data
print(len(list(dataset.train)))
print(len(list(dataset.test)))

entry = list(dataset.test)[0]

print("entry:\n", entry)
print("metadata: \n", dataset.metadata)
print("prediction_length:n", dataset.metadata.prediction_length)

INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\traffic.


862
6034
entry:
 {'start': Timestamp('2015-01-01 00:00:00', freq='H'), 'target': array([0.0048, 0.0072, 0.004 , ..., 0.0467, 0.0412, 0.0386], dtype=float32), 'feat_static_cat': array([0]), 'source': SourceContext(source=Span(path=WindowsPath('C:/Users/TM/.mxnet/gluon-ts/datasets/traffic/test/data.json'), line=1), row=1)}
metadata: 
 MetaData freq='H' target=None feat_static_cat=[<CategoricalFeatureInfo name='feat_static_cat' cardinality='862'>] feat_static_real=[] feat_dynamic_real=[] feat_dynamic_cat=[] prediction_length=24
prediction_length:n 24


### Run 1: Traffic

In [5]:
dataset = get_dataset("traffic")
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,      
#         batch_size=32,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=28,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\traffic.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 30683
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.72it/s, avg_epoch_loss=-2.36]
INFO:root:Epoch[0] Elapsed time 13.495 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-2.358730
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.31it/s, avg_epoch_loss=-3.16]
INFO:root:Epoch[1] Elapsed time 11.613 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=-3.158141
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.48it/s, avg_epoch_loss=-3.37]
INFO:root:Epoch[2] Elapsed time 11.170 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-3.366660
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  4.01it/s, avg_epoch_loss=-3.41]
INFO:root:Epoch[3] Elapsed time 12.485 se

INFO:root:Epoch[72] Elapsed time 14.088 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=-3.982753
INFO:root:Epoch[73] Learning rate is 0.0005
100%|██████████| 50/50 [00:11<00:00,  4.17it/s, avg_epoch_loss=-3.98]
INFO:root:Epoch[73] Elapsed time 11.987 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=-3.982261
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:12<00:00,  3.88it/s, avg_epoch_loss=-3.99]
INFO:root:Epoch[74] Elapsed time 12.886 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=-3.990227
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:15<00:00,  3.13it/s, avg_epoch_loss=-4.01]
INFO:root:Epoch[75] Elapsed time 15.966 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=-4.006844
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:16<00:00,  3.11it/s, avg_epoch_loss=-4]   
INFO:root:Epoch[76] Elapsed time 16.074 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=-4.000790

{'Coverage[0.1]': 0.06697760310620829,
 'Coverage[0.2]': 0.16610634973246866,
 'Coverage[0.3]': 0.28133434348216935,
 'Coverage[0.4]': 0.40196032009091576,
 'Coverage[0.5]': 0.5232255315119034,
 'Coverage[0.6]': 0.6223779535015793,
 'Coverage[0.7]': 0.7243003930110223,
 'Coverage[0.8]': 0.8154742175292276,
 'Coverage[0.9]': 0.8976750793124585,
 'MAE_Coverage': 0.019471671112371514,
 'MASE': 0.7977977865211145,
 'MSE': 0.0009014114883876248,
 'MSIS': 10.121165065568231,
 'ND': 0.1522114584595843,
 'NRMSE': 0.3473113802315723,
 'QuantileLoss[0.1]': 239.985514632304,
 'QuantileLoss[0.2]': 370.10992455042435,
 'QuantileLoss[0.3]': 461.40128506007454,
 'QuantileLoss[0.4]': 522.0782269757591,
 'QuantileLoss[0.5]': 555.7675978703192,
 'QuantileLoss[0.6]': 563.2912781853229,
 'QuantileLoss[0.7]': 537.3710571142379,
 'QuantileLoss[0.8]': 475.58436708860097,
 'QuantileLoss[0.9]': 359.1258022917667,
 'RMSE': 0.030023515590077467,
 'abs_error': 555.767598522827,
 'abs_target_mean': 0.0864455278432

### Run 2: Traffic

In [6]:
dataset = get_dataset("traffic")
mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,      
#         batch_size=32,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=28,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\traffic.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 30683
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.27it/s, avg_epoch_loss=-2.43]
INFO:root:Epoch[0] Elapsed time 11.727 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-2.426429
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.72it/s, avg_epoch_loss=-3.16]
INFO:root:Epoch[1] Elapsed time 10.593 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=-3.155029
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.42it/s, avg_epoch_loss=-3.38]
INFO:root:Epoch[2] Elapsed time 11.325 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-3.381457
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.64it/s, avg_epoch_loss=-3.47]
INFO:root:Epoch[3] Elapsed time 10.775 se

100%|██████████| 50/50 [00:10<00:00,  4.63it/s, avg_epoch_loss=-3.9] 
INFO:root:Epoch[36] Elapsed time 10.798 seconds
INFO:root:Epoch[36] Evaluation metric 'epoch_loss'=-3.902633
INFO:root:Epoch[37] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.62it/s, avg_epoch_loss=-3.9] 
INFO:root:Epoch[37] Elapsed time 10.817 seconds
INFO:root:Epoch[37] Evaluation metric 'epoch_loss'=-3.900836
INFO:root:Epoch[38] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.52it/s, avg_epoch_loss=-3.91]
INFO:root:Epoch[38] Elapsed time 11.060 seconds
INFO:root:Epoch[38] Evaluation metric 'epoch_loss'=-3.913886
INFO:root:Epoch[39] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.69it/s, avg_epoch_loss=-3.88]
INFO:root:Epoch[39] Elapsed time 10.674 seconds
INFO:root:Epoch[39] Evaluation metric 'epoch_loss'=-3.875704
INFO:root:Epoch[40] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.60it/s, avg_epoch_loss=-3.89]
INFO:root:Epoch[40] Elapsed time 10.883 se

{'Coverage[0.1]': 0.07112079170415349,
 'Coverage[0.2]': 0.16686396136180773,
 'Coverage[0.3]': 0.2850987262654458,
 'Coverage[0.4]': 0.4059141057815229,
 'Coverage[0.5]': 0.525829821487755,
 'Coverage[0.6]': 0.6278469624508631,
 'Coverage[0.7]': 0.727354514891792,
 'Coverage[0.8]': 0.8169657654244913,
 'Coverage[0.9]': 0.8963729343245332,
 'MAE_Coverage': 0.02049497293116491,
 'MASE': 0.7830419072121775,
 'MSE': 0.0008918347815035373,
 'MSIS': 10.199148544469137,
 'ND': 0.14943473552889816,
 'NRMSE': 0.34546151433063,
 'QuantileLoss[0.1]': 238.87178373606656,
 'QuantileLoss[0.2]': 364.3919455388095,
 'QuantileLoss[0.3]': 452.35013306723704,
 'QuantileLoss[0.4]': 512.5938663023059,
 'QuantileLoss[0.5]': 545.6289875030925,
 'QuantileLoss[0.6]': 554.2627124581719,
 'QuantileLoss[0.7]': 530.7888443737988,
 'QuantileLoss[0.8]': 470.7220105820685,
 'QuantileLoss[0.9]': 353.4913405695813,
 'RMSE': 0.029863602955831323,
 'abs_error': 545.6289883907884,
 'abs_target_mean': 0.08644552784322551,

### Run 3: Traffic

In [7]:
dataset = get_dataset("traffic")
mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,     
#         batch_size=32,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=28,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\traffic.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 30683
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.23it/s, avg_epoch_loss=-2.38]
INFO:root:Epoch[0] Elapsed time 11.816 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=-2.380284
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.78it/s, avg_epoch_loss=-3.16]
INFO:root:Epoch[1] Elapsed time 13.244 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=-3.162003
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  3.86it/s, avg_epoch_loss=-3.36]
INFO:root:Epoch[2] Elapsed time 12.950 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=-3.358579
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:16<00:00,  3.12it/s, avg_epoch_loss=-3.47]
INFO:root:Epoch[3] Elapsed time 16.017 se

100%|██████████| 50/50 [00:14<00:00,  3.52it/s, avg_epoch_loss=-3.96]
INFO:root:Epoch[72] Elapsed time 14.223 seconds
INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=-3.964101
INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.77it/s, avg_epoch_loss=-3.99]
INFO:root:Epoch[73] Elapsed time 13.285 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=-3.989371
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.79it/s, avg_epoch_loss=-3.95]
INFO:root:Epoch[74] Elapsed time 13.193 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=-3.946432
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 50/50 [00:12<00:00,  3.93it/s, avg_epoch_loss=-3.96]
INFO:root:Epoch[75] Elapsed time 12.726 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=-3.956946
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 50/50 [00:12<00:00,  3.94it/s, avg_epoch_loss=-3.99]
INFO:root:Epoch[76] Elapsed time 1

{'Coverage[0.1]': 0.05331691841469799,
 'Coverage[0.2]': 0.13246365831715598,
 'Coverage[0.3]': 0.2296036744163996,
 'Coverage[0.4]': 0.34239310573417203,
 'Coverage[0.5]': 0.4674937260286906,
 'Coverage[0.6]': 0.5817273545148836,
 'Coverage[0.7]': 0.6995359628770191,
 'Coverage[0.8]': 0.8042047445428188,
 'Coverage[0.9]': 0.8970595198636192,
 'MAE_Coverage': 0.03340120270846451,
 'MASE': 0.8161954964811612,
 'MSE': 0.000930536504411243,
 'MSIS': 9.99008703155535,
 'ND': 0.1560744120244462,
 'NRMSE': 0.35287766982425756,
 'QuantileLoss[0.1]': 254.04680529026527,
 'QuantileLoss[0.2]': 386.09419590118637,
 'QuantileLoss[0.3]': 477.6626300807724,
 'QuantileLoss[0.4]': 537.6981718753144,
 'QuantileLoss[0.5]': 569.8723469593679,
 'QuantileLoss[0.6]': 576.263417582214,
 'QuantileLoss[0.7]': 549.4800107376418,
 'QuantileLoss[0.8]': 483.77943030853055,
 'QuantileLoss[0.9]': 359.13621058515275,
 'RMSE': 0.030504696432045394,
 'abs_error': 569.8723475849256,
 'abs_target_mean': 0.086445527843225

### Run 1 - electricity

In [13]:
dataset = get_dataset("electricity")
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,     
#         batch_size=32,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\electricity.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 19863
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:59<00:00,  1.20s/it, avg_epoch_loss=6.8] 
INFO:root:Epoch[0] Elapsed time 60.116 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.796252
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [01:08<00:00,  1.37s/it, avg_epoch_loss=5.88]
INFO:root:Epoch[1] Elapsed time 68.452 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=5.875404
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [01:08<00:00,  1.37s/it, avg_epoch_loss=5.72]
INFO:root:Epoch[2] Elapsed time 68.491 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=5.715046
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [01:15<00:00,  1.52s/it, avg_epoch_loss=5.66]
INFO:root:Epoch[3] Elapsed time 75.970 secon

INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=5.046485
INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 50/50 [00:37<00:00,  1.34it/s, avg_epoch_loss=5.07]
INFO:root:Epoch[73] Elapsed time 37.419 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.073239
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 50/50 [01:14<00:00,  1.49s/it, avg_epoch_loss=5.08]
INFO:root:Epoch[74] Elapsed time 74.769 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.080558
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 50/50 [00:54<00:00,  1.09s/it, avg_epoch_loss=5.13]
INFO:root:Epoch[75] Elapsed time 54.408 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.126575
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 50/50 [00:46<00:00,  1.08it/s, avg_epoch_loss=5.07]
INFO:root:Epoch[76] Elapsed time 46.126 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.066138
INFO:root:Epoch[77] Learning rate is 0.00025
100%|██

{'Coverage[0.1]': 0.07552927713141319,
 'Coverage[0.2]': 0.14063195371606646,
 'Coverage[0.3]': 0.21215589039354196,
 'Coverage[0.4]': 0.28526924788607116,
 'Coverage[0.5]': 0.3685548985949533,
 'Coverage[0.6]': 0.4520312798016419,
 'Coverage[0.7]': 0.5594761268993568,
 'Coverage[0.8]': 0.6732786572572981,
 'Coverage[0.9]': 0.7995422468052685,
 'MAE_Coverage': 0.10372560239048763,
 'MASE': 0.8341907587028697,
 'MSE': 4846164.124408975,
 'MSIS': 10.815703975750113,
 'ND': 0.07130017816651193,
 'NRMSE': 0.682200467615783,
 'QuantileLoss[0.1]': 1382726.2558807633,
 'QuantileLoss[0.2]': 2254335.6072767647,
 'QuantileLoss[0.3]': 2891924.56603128,
 'QuantileLoss[0.4]': 3338986.780232748,
 'QuantileLoss[0.5]': 3618918.2434031814,
 'QuantileLoss[0.6]': 3776671.0316948947,
 'QuantileLoss[0.7]': 3691938.648503849,
 'QuantileLoss[0.8]': 3390618.353511584,
 'QuantileLoss[0.9]': 2682583.050326887,
 'RMSE': 2201.4004915982405,
 'abs_error': 3618918.260035515,
 'abs_target_mean': 3226.911437472181,
 

### Run 2 - electricity

In [14]:
dataset = get_dataset("electricity")
mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,     
#         batch_size=32,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\electricity.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 19863
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:28<00:00,  1.73it/s, avg_epoch_loss=6.38]
INFO:root:Epoch[0] Elapsed time 29.129 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.381408
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:26<00:00,  1.89it/s, avg_epoch_loss=5.78]
INFO:root:Epoch[1] Elapsed time 26.465 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=5.780199
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:23<00:00,  2.10it/s, avg_epoch_loss=5.63]
INFO:root:Epoch[2] Elapsed time 23.782 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=5.631609
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:25<00:00,  1.96it/s, avg_epoch_loss=5.62]
INFO:root:Epoch[3] Elapsed time 25.463 secon

INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=5.087863
INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.81it/s, avg_epoch_loss=5.04]
INFO:root:Epoch[73] Elapsed time 13.121 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.041262
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.82it/s, avg_epoch_loss=5.11]
INFO:root:Epoch[74] Elapsed time 13.094 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.108409
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.63it/s, avg_epoch_loss=5.09]
INFO:root:Epoch[75] Elapsed time 13.789 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.089559
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.80it/s, avg_epoch_loss=5.06]
INFO:root:Epoch[76] Elapsed time 13.166 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.057352
INFO:root:Epoch[77] Learning rate is 0.00025
100%|██

{'Coverage[0.1]': 0.06898086337338659,
 'Coverage[0.2]': 0.13166762031915613,
 'Coverage[0.3]': 0.19696102740161603,
 'Coverage[0.4]': 0.2726810350308351,
 'Coverage[0.5]': 0.3528514209422094,
 'Coverage[0.6]': 0.43550130332506953,
 'Coverage[0.7]': 0.5420560747663551,
 'Coverage[0.8]': 0.664441477525592,
 'Coverage[0.9]': 0.7965541356729653,
 'MAE_Coverage': 0.11536722684920164,
 'MASE': 0.8472640926425936,
 'MSE': 4347845.535482423,
 'MSIS': 10.705347455994795,
 'ND': 0.06915737141209485,
 'NRMSE': 0.6461747849490488,
 'QuantileLoss[0.1]': 1308437.9114143888,
 'QuantileLoss[0.2]': 2148589.408917986,
 'QuantileLoss[0.3]': 2773215.0149978935,
 'QuantileLoss[0.4]': 3221019.2319817636,
 'QuantileLoss[0.5]': 3510157.7464532796,
 'QuantileLoss[0.6]': 3663579.588078048,
 'QuantileLoss[0.7]': 3610441.3922873493,
 'QuantileLoss[0.8]': 3299292.354582173,
 'QuantileLoss[0.9]': 2593244.223709812,
 'RMSE': 2085.1488041582124,
 'abs_error': 3510157.7675557137,
 'abs_target_mean': 3226.911437472181

### Run 3 - electricity

In [15]:
dataset = get_dataset("electricity")
mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,     
#         batch_size=32,      
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=21,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=7,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\electricity.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 19863
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.59it/s, avg_epoch_loss=6.94]
INFO:root:Epoch[0] Elapsed time 13.926 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=6.941189
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:14<00:00,  3.37it/s, avg_epoch_loss=5.93]
INFO:root:Epoch[1] Elapsed time 14.839 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=5.931306
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:13<00:00,  3.65it/s, avg_epoch_loss=5.76]
INFO:root:Epoch[2] Elapsed time 13.705 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=5.759417
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:14<00:00,  3.49it/s, avg_epoch_loss=5.63]
INFO:root:Epoch[3] Elapsed time 14.352 secon

INFO:root:Epoch[73] Learning rate is 0.00025
100%|██████████| 50/50 [00:12<00:00,  3.92it/s, avg_epoch_loss=5.11]
INFO:root:Epoch[73] Elapsed time 12.789 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=5.112404
INFO:root:Epoch[74] Learning rate is 0.00025
100%|██████████| 50/50 [00:12<00:00,  4.01it/s, avg_epoch_loss=5.12]
INFO:root:Epoch[74] Elapsed time 12.459 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=5.123464
INFO:root:Epoch[75] Learning rate is 0.00025
100%|██████████| 50/50 [00:12<00:00,  3.92it/s, avg_epoch_loss=5.16]
INFO:root:Epoch[75] Elapsed time 12.777 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=5.156730
INFO:root:Epoch[76] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.69it/s, avg_epoch_loss=5.13]
INFO:root:Epoch[76] Elapsed time 13.551 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=5.127476
INFO:root:Epoch[77] Learning rate is 0.00025
100%|██████████| 50/50 [00:13<00:00,  3.85it/s, avg_epoch_loss=5.07

{'Coverage[0.1]': 0.07368554898594935,
 'Coverage[0.2]': 0.15080424693241828,
 'Coverage[0.3]': 0.23250047682624583,
 'Coverage[0.4]': 0.3237332316104018,
 'Coverage[0.5]': 0.41814482802466824,
 'Coverage[0.6]': 0.5157988429016471,
 'Coverage[0.7]': 0.6262318011316697,
 'Coverage[0.8]': 0.7393985631635867,
 'Coverage[0.9]': 0.8536461313497412,
 'MAE_Coverage': 0.06289514767485244,
 'MASE': 0.7966944083115938,
 'MSE': 3694232.276017577,
 'MSIS': 9.217468875198767,
 'ND': 0.06519956931542852,
 'NRMSE': 0.5956279278876739,
 'QuantileLoss[0.1]': 1370549.6424202877,
 'QuantileLoss[0.2]': 2175208.085820111,
 'QuantileLoss[0.3]': 2730710.234585321,
 'QuantileLoss[0.4]': 3097410.352756396,
 'QuantileLoss[0.5]': 3309275.206174981,
 'QuantileLoss[0.6]': 3380974.8604001394,
 'QuantileLoss[0.7]': 3240012.766428671,
 'QuantileLoss[0.8]': 2890758.2603530227,
 'QuantileLoss[0.9]': 2151582.1735818572,
 'RMSE': 1922.0385729785905,
 'abs_error': 3309275.208135128,
 'abs_target_mean': 3226.911437472181,


In [11]:
dataset = get_dataset("m4_hourly")

# Inspect data
print(len(list(dataset.train)))
print(len(list(dataset.test)))

entry = list(dataset.test)[16]

print("entry:\n", entry)
print("metadata: \n", dataset.metadata)
print("prediction_length:n", dataset.metadata.prediction_length)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.


414
414
entry:
 {'start': Timestamp('1750-01-01 00:00:00', freq='H'), 'target': array([52694., 50487., 48905., 47791., 44692., 40769., 37456., 35119.,
       33638., 32798., 32805., 34121., 35894., 37403., 39964., 43192.,
       46593., 48829., 50231., 51645., 52622., 53774., 53665., 52925.,
       51756., 50171., 48655., 47667., 44629., 40754., 37547., 35199.,
       33637., 32759., 32536., 33527., 35196., 36637., 39221., 42398.,
       45675., 48667., 51260., 53719., 55525., 56836., 57505., 56807.,
       55044., 52776., 50986., 49845., 46817., 43307., 40173., 37954.,
       36353., 35346., 34981., 35402., 35919., 36447., 38820., 42009.,
       45201., 48116., 50501., 52232., 53443., 54180., 54510., 54028.,
       52732., 50673., 49028., 47940., 45385., 42470., 39566., 37379.,
       35757., 34698., 34057., 33939., 33910., 34456., 37333., 41075.,
       45018., 48470., 51173., 52980., 54226., 54755., 54722., 54308.,
       52974., 50742., 48474., 46782., 44767., 42872., 40437., 38209

## Data: `m4_hourly`

#### Run 1

In [8]:
dataset = get_dataset("m4_hourly")
mx.random.seed(42)
np.random.seed(42)

if __name__ == "__main__":
    
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,        
        num_batches_per_epoch=50,     
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=48,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=48,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.49it/s, avg_epoch_loss=5.28]
INFO:root:Epoch[0] Elapsed time 11.152 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.280023
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.69it/s, avg_epoch_loss=4.52]
INFO:root:Epoch[1] Elapsed time 10.669 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.521267
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.53it/s, avg_epoch_loss=4.16]
INFO:root:Epoch[2] Elapsed time 9.048 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.156798
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.30it/s, avg_epoch_loss=3.66]
INFO:root:Epoch[3] Elapsed time 9.443 seconds
I

INFO:root:Epoch[73] Elapsed time 9.982 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.648038
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:10<00:00,  4.82it/s, avg_epoch_loss=2.88]
INFO:root:Epoch[74] Elapsed time 10.375 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.881696
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.11it/s, avg_epoch_loss=2.68]
INFO:root:Epoch[75] Elapsed time 9.784 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.684727
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.24it/s, avg_epoch_loss=2.82]
INFO:root:Epoch[76] Elapsed time 9.546 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.817950
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.05it/s, avg_epoch_loss=2.83]
INFO:root:Epoch[77] Elapsed time 9.917 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.833692
INFO:root:Ep

{'Coverage[0.1]': 0.2193035426731077,
 'Coverage[0.2]': 0.3049013687600642,
 'Coverage[0.3]': 0.3820954106280194,
 'Coverage[0.4]': 0.4523953301127216,
 'Coverage[0.5]': 0.5211855877616749,
 'Coverage[0.6]': 0.580968196457327,
 'Coverage[0.7]': 0.6506139291465377,
 'Coverage[0.8]': 0.7207628824476646,
 'Coverage[0.9]': 0.8006239935587769,
 'MAE_Coverage': 0.06965691536947573,
 'MASE': 1.339494444687416,
 'MSE': 4183684.922365106,
 'MSIS': 21.736033959710518,
 'ND': 0.048259866751103414,
 'NRMSE': 0.27924307201782844,
 'QuantileLoss[0.1]': 4014071.236398793,
 'QuantileLoss[0.2]': 5449269.579226495,
 'QuantileLoss[0.3]': 6326800.884770488,
 'QuantileLoss[0.4]': 6812774.4857089985,
 'QuantileLoss[0.5]': 7024651.359565735,
 'QuantileLoss[0.6]': 6920977.141727828,
 'QuantileLoss[0.7]': 6542560.461486817,
 'QuantileLoss[0.8]': 5829813.100245284,
 'QuantileLoss[0.9]': 4646873.226919174,
 'RMSE': 2045.4058087247886,
 'abs_error': 7024651.361759186,
 'abs_target_mean': 7324.822041043147,
 'abs_

#### Run 2

In [9]:
dataset = get_dataset("m4_hourly")
mx.random.seed(43)
np.random.seed(43)

if __name__ == "__main__":
    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,           
        num_batches_per_epoch=50,      
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=48,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=48,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:12<00:00,  4.11it/s, avg_epoch_loss=5.52]
INFO:root:Epoch[0] Elapsed time 12.163 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.516625
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.96it/s, avg_epoch_loss=4.58]
INFO:root:Epoch[1] Elapsed time 10.089 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.577677
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.01it/s, avg_epoch_loss=4.08]
INFO:root:Epoch[2] Elapsed time 9.997 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.082698
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.98it/s, avg_epoch_loss=3.77]
INFO:root:Epoch[3] Elapsed time 10.056 seconds


INFO:root:Epoch[72] Evaluation metric 'epoch_loss'=2.855379
INFO:root:Epoch[73] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.34it/s, avg_epoch_loss=2.9]
INFO:root:Epoch[73] Elapsed time 9.363 seconds
INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.904866
INFO:root:Epoch[74] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.37it/s, avg_epoch_loss=2.93]
INFO:root:Epoch[74] Elapsed time 9.326 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.926818
INFO:root:Epoch[75] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.26it/s, avg_epoch_loss=2.9]
INFO:root:Epoch[75] Elapsed time 9.517 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.903587
INFO:root:Epoch[76] Learning rate is 0.000125
100%|██████████| 50/50 [00:09<00:00,  5.07it/s, avg_epoch_loss=2.98]
INFO:root:Epoch[76] Elapsed time 9.874 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.979229
INFO:root:Loading parameters from best epoch (66)
INFO

{'Coverage[0.1]': 0.19439412238325268,
 'Coverage[0.2]': 0.26494565217391286,
 'Coverage[0.3]': 0.32981078904991945,
 'Coverage[0.4]': 0.3883856682769727,
 'Coverage[0.5]': 0.4466586151368762,
 'Coverage[0.6]': 0.502969001610306,
 'Coverage[0.7]': 0.5683373590982284,
 'Coverage[0.8]': 0.6360708534621576,
 'Coverage[0.9]': 0.7220712560386464,
 'MAE_Coverage': 0.09162864555376642,
 'MASE': 1.4937566648691005,
 'MSE': 7387000.336408353,
 'MSIS': 27.869067055412792,
 'ND': 0.06411661686083005,
 'NRMSE': 0.3710538837040973,
 'QuantileLoss[0.1]': 8834856.205185318,
 'QuantileLoss[0.2]': 9985400.288142204,
 'QuantileLoss[0.3]': 10237161.856968973,
 'QuantileLoss[0.4]': 9972057.491693115,
 'QuantileLoss[0.5]': 9332741.979476929,
 'QuantileLoss[0.6]': 8346155.960227966,
 'QuantileLoss[0.7]': 7077066.434729195,
 'QuantileLoss[0.8]': 5492653.74414978,
 'QuantileLoss[0.9]': 3493867.861057472,
 'RMSE': 2717.9036657704323,
 'abs_error': 9332741.888113976,
 'abs_target_mean': 7324.822041043147,
 'abs

#### Run 3

In [10]:
dataset = get_dataset("m4_hourly")
mx.random.seed(44)
np.random.seed(44)

if __name__ == "__main__":

    trainer = Trainer(
        ctx=mx.cpu(0),
        epochs=100,        
        num_batches_per_epoch=50,     
        learning_rate=1e-3,
    )

    cardinality = int(dataset.metadata.feat_static_cat[0].cardinality)
    estimator = DeepAREstimator(
        trainer=trainer,
        context_length=48,
        use_feat_static_cat=True,
        cardinality=[cardinality],
        prediction_length=48,
        freq=dataset.metadata.freq,
    )

    predictor = estimator.train(dataset.train)
    
    forecast_it, ts_it = make_evaluation_predictions(
    dataset.test, predictor=predictor, num_eval_samples=100
    )
    
    agg_metrics, item_metrics = Evaluator()(
    ts_it, forecast_it, num_series=len(dataset.test)
    )
    
    pprint(agg_metrics)


INFO:root:using dataset already processed in path C:\Users\TM\.mxnet\gluon-ts\datasets\m4_hourly.
INFO:root:Start model training
INFO:root:Number of parameters in DeepARTrainingNetwork: 21723
INFO:root:Epoch[0] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.63it/s, avg_epoch_loss=5.11]
INFO:root:Epoch[0] Elapsed time 10.807 seconds
INFO:root:Epoch[0] Evaluation metric 'epoch_loss'=5.111396
INFO:root:Epoch[1] Learning rate is 0.001
100%|██████████| 50/50 [00:09<00:00,  5.00it/s, avg_epoch_loss=4.26]
INFO:root:Epoch[1] Elapsed time 10.004 seconds
INFO:root:Epoch[1] Evaluation metric 'epoch_loss'=4.256519
INFO:root:Epoch[2] Learning rate is 0.001
100%|██████████| 50/50 [00:11<00:00,  4.48it/s, avg_epoch_loss=4.27]
INFO:root:Epoch[2] Elapsed time 11.181 seconds
INFO:root:Epoch[2] Evaluation metric 'epoch_loss'=4.270225
INFO:root:Epoch[3] Learning rate is 0.001
100%|██████████| 50/50 [00:10<00:00,  4.69it/s, avg_epoch_loss=4.05]
INFO:root:Epoch[3] Elapsed time 10.672 seconds

INFO:root:Epoch[73] Evaluation metric 'epoch_loss'=2.948231
INFO:root:Loading parameters from best epoch (63)
INFO:root:Epoch[74] Learning rate is 0.0005
100%|██████████| 50/50 [00:11<00:00,  4.44it/s, avg_epoch_loss=2.92]
INFO:root:Epoch[74] Elapsed time 11.270 seconds
INFO:root:Epoch[74] Evaluation metric 'epoch_loss'=2.916431
INFO:root:Epoch[75] Learning rate is 0.0005
100%|██████████| 50/50 [00:10<00:00,  4.83it/s, avg_epoch_loss=2.89]
INFO:root:Epoch[75] Elapsed time 10.349 seconds
INFO:root:Epoch[75] Evaluation metric 'epoch_loss'=2.886410
INFO:root:Epoch[76] Learning rate is 0.0005
100%|██████████| 50/50 [00:09<00:00,  5.30it/s, avg_epoch_loss=2.45]
INFO:root:Epoch[76] Elapsed time 9.440 seconds
INFO:root:Epoch[76] Evaluation metric 'epoch_loss'=2.453312
INFO:root:Epoch[77] Learning rate is 0.0005
100%|██████████| 50/50 [00:10<00:00,  4.93it/s, avg_epoch_loss=2.95]
INFO:root:Epoch[77] Elapsed time 10.153 seconds
INFO:root:Epoch[77] Evaluation metric 'epoch_loss'=2.951038
INFO:ro

{'Coverage[0.1]': 0.2977053140096616,
 'Coverage[0.2]': 0.3756541867954912,
 'Coverage[0.3]': 0.4387580515297907,
 'Coverage[0.4]': 0.493961352657005,
 'Coverage[0.5]': 0.5434782608695653,
 'Coverage[0.6]': 0.5849939613526569,
 'Coverage[0.7]': 0.6347624798711753,
 'Coverage[0.8]': 0.6896135265700485,
 'Coverage[0.9]': 0.7486413043478265,
 'MAE_Coverage': 0.11017176596886741,
 'MASE': 2.162661147215832,
 'MSE': 51197135.181636654,
 'MSIS': 47.482280539307204,
 'ND': 0.13483371425025703,
 'NRMSE': 0.9768452120525684,
 'QuantileLoss[0.1]': 5642416.087568759,
 'QuantileLoss[0.2]': 9797066.959729765,
 'QuantileLoss[0.3]': 13430605.929391384,
 'QuantileLoss[0.4]': 16704168.73035393,
 'QuantileLoss[0.5]': 19626242.385269165,
 'QuantileLoss[0.6]': 22246088.176530074,
 'QuantileLoss[0.7]': 24401930.14681969,
 'QuantileLoss[0.8]': 25980818.0787632,
 'QuantileLoss[0.9]': 26527825.87313557,
 'RMSE': 7155.217339930119,
 'abs_error': 19626242.22118187,
 'abs_target_mean': 7324.822041043147,
 'abs_t

In [6]:
# import rpy2
# print(rpy2.__version__)

# # importing packages
# from rpy2.robjects.packages import importr
# import rpy2.robjects.packages as rpackages

# base=importr("base")
# utils=importr("utils")

# utils = rpackages.importr("utils")
# utils.chooseCRANmirror(ind=1)

# utils.install_packages("forecast")
# utils.install_packages("nnfor")